# Movie Content-Based Recommendation Engine

# About

In this notebook, I will go through building a content-based recommendation engine based on several movies that I've enjoyed in the past. I love movies, however, I have not had a chance to see many of the classics of the past, or get overwhelmed with how many good ones keep popping up each year and slipping under my radar.

Luckily, with a content-based recommendation engine, I can get suggests as to what I might like to watch next based on the movies I've liked in the past and what movies I'd rather stay away from (I'm looking at you, Twilight Saga).

# Dataset

The dataset of interest is an IMDB TOP 250 English movies, a fairly well-known dataset

## Download and install necessary packages

One of the more unique libraries that we might work with for natural language processing is the Rapid Automatic Keyword Extraction - natural language toolkit, or rake-nltk for short. We will have to install it using pip commandline argument below.

In [0]:
!pip install rake_nltk -qq
!pip install pandas-profiling -qq

## Load Dependencies

In [0]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os, re
from timeit import default_timer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import gensim
from google.colab import drive
import pandas_profiling

## Load Data

In [3]:
DRIVENAME = "/content/drive/"
FILENAME = "/content/drive/My Drive/problem_set_challenges_for_job_postings/movie_metadata.csv"
drive.mount(DRIVENAME)
df = pd.read_csv(FILENAME, header="infer")
df.head()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.0,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1.0,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,Tom Hardy,The Dark Knight Rises,1144337,106759,Joseph Gordon-Levitt,0.0,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens ...,8,143,NaN,0.0,NaN,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


## Quick Pandas Profiling

In [4]:
report = pandas_profiling.ProfileReport(df)
report

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,28
Number of observations,5043
Total Missing (%),1.9%
Total size in memory,1.1 MiB
Average record size in memory,224.0 B
Numeric,15
Categorical,12
Boolean,0
Date,0
Text (Unique),0
Rejected,1


Notes:

* 

## Quick Numeric Histogram plot

In [5]:
pd.DataFrame.hist(df._get_numeric_data(), figsize=(15, 15))

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7f4a94dc3390>,
      dtype=object)

## Dropping Data

### Drop Highly correlated data

In [0]:
correlated_variables = report.get_rejected_variables()
df.drop(correlated_variables, axis=1, inplace=True)

In [0]:
report.to_file(outputfile="imdb_top250_eng_movies_pandasprofiling.html")

Have a look at the movies with more than 0 Facebook likes

In [8]:
morethanzeroFBlikes_df = df.loc[df.movie_facebook_likes>0, :]
morethanzeroFBlikes_df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,Stephanie Sigman,1.0,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,Tom Hardy,The Dark Knight Rises,1144337,Joseph Gordon-Levitt,0.0,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
5,Color,Andrew Stanton,462.0,132.0,475.0,530.0,Samantha Morton,640.0,73058679.0,Action|Adventure|Sci-Fi,Daryl Sabara,John Carter,212204,Polly Walker,1.0,alien|american civil war|male nipple|mars|prin...,http://www.imdb.com/title/tt0401729/?ref_=fn_t...,738.0,English,USA,PG-13,263700000.0,2012.0,632.0,6.6,2.35,24000
7,Color,Nathan Greno,324.0,100.0,15.0,284.0,Donna Murphy,799.0,200807262.0,Adventure|Animation|Comedy|Family|Fantasy|Musi...,Brad Garrett,Tangled,294810,M.C. Gainey,1.0,17th century|based on fairy tale|disney|flower...,http://www.imdb.com/title/tt0398286/?ref_=fn_t...,387.0,English,USA,PG,260000000.0,2010.0,553.0,7.8,1.85,29000


In [0]:
# df['director_name'].value_counts().sort_values(ascending=False)[0:10].plot(kind="barh", 
#                                                                            figsize=(8,5), 
#                                                                            fontsize=15, 
#                                                                            color="navy").invert_yaxis()

## Define Preprocessing Functions

In [0]:
def listify_column_values(dataframe, cols):
    """ removes bars from string and listifies it. make sure that there are no NaN values in the series first """
    for col in cols:
        if dataframe[col].isnull().sum()>0:
            dataframe[col].dropna(axis=0, inplace=True)
            dataframe[col] = dataframe[col].apply(lambda x: x.split("|"))
        else:
            dataframe[col] = dataframe[col].apply(lambda x: x.split("|"))

def normalize_entity(dataframe, cols):
    """ make sure that there are no NaN values in the series first """
    for col in cols:
        if dataframe[col].isnull().sum()>0:
            dataframe[col].dropna(axis=0, inplace=True)
            dataframe[col] = dataframe[col].apply(lambda x: x.lower().replace(" ", ""))
        else:
            dataframe[col] = dataframe[col].apply(lambda x: x.lower().replace(" ", ""))

In [0]:
cols_to_listify = ["genres", "plot_keywords"]
listify_column_values(df, cols_to_listify)

In [0]:
actor_cols = ["director_name", "actor_1_name", 'actor_2_name', "actor_3_name"]
normalize_entity(df, actor_cols)

In [0]:
df["actor_1_name"].dropna(axis=0, inplace=True)
df["actor_2_name"].dropna(axis=0, inplace=True)
df["actor_3_name"].dropna(axis=0, inplace=True)

In [0]:
df["actors"] = df["actor_1_name"].combine(df["actor_2_name"], lambda x, y: [x,y])
df["actors"] = df["actors"].combine(df["actor_3_name"], lambda x, y: [x, y] if not pd.isnull(y) else x)

In [0]:
def flatten_to_strings(listOfLists):
    """Flatten a list of (lists of (lists of strings)) for any level 
    of nesting"""
    result = []

    for i in listOfLists:
        # Only append if i is a basestring (superclass of string)
        if isinstance(i, str):
            result.append(str(i))
        # Otherwise call this function recursively
        else:
            for x in i:
                result.append(str(x))
    return result

In [0]:
df["actors"] = df["actors"].apply(lambda x: flatten_to_strings(list(x)))
df["actors"]

In [0]:
df['bag_of_words'] = ""
columns = ["genres", "director_name", "actors"]
for index, row in df.iterrows():
    words = ""
    for col in columns:
        words += " ".join((row[col])) + " "
        row["bag_of_words"] = words 
ef = df[["movie_title", "bag_of_words"]]
ef.head()

## Sandbox layer

In [0]:
super_list = c[0]
f = list(set([item if isinstance(sublist, list) else sublist for sublist in super_list for item in sublist]))
g = [item if isinstance(sublist, list) else sublist for sublist in super_list for item in sublist]
# [f(x) if condition else g(x) for x in sequence]
g